## 1. Import Library yang dibutuhkan

Konfigurasi perangkat (menggunakan GPU jika tersedia, jika tidak menggunakan CPU)

In [47]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('isAvailable: ', device)

isAvailable:  cuda


In [48]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


## 2. Hyperparameter

In [49]:
num_epochs = 100
learning_rate = 0.001
batch_size = 128

## 3. Preprocessing data dan augmentasi

In [50]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # Augmentasi data (random cropping)
    transforms.RandomHorizontalFlip(),     # Augmentasi data (flip horizontal)
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalisasi data
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalisasi data uji
])

## 4. Load dataset CIFAR-10

In [51]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


## 5.  Model Convolutional Neural Networks (CNN)

In [52]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # Konvolusi pertama
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Konvolusi kedua
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # Konvolusi ketiga
        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)  # Konvolusi keempat
        
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling
        self.dropout = nn.Dropout(0.5)  # Dropout dengan probabilitas 0.5
        
        self.fc1 = nn.Linear(512 * 2 * 2, 1024)  # Fully connected layer pertama
        self.fc2 = nn.Linear(1024, 512)  # Fully connected layer kedua
        self.fc3 = nn.Linear(512, 10)  # Fully connected layer untuk output (10 kelas)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Konvolusi pertama + ReLU + MaxPool
        x = self.pool(F.relu(self.conv2(x)))  # Konvolusi kedua + ReLU + MaxPool
        x = self.pool(F.relu(self.conv3(x)))  # Konvolusi ketiga + ReLU + MaxPool
        x = self.pool(F.relu(self.conv4(x)))  # Konvolusi keempat + ReLU + MaxPool
        
        x = x.view(-1, 512 * 2 * 2)  # Flatten fitur map
        
        x = F.relu(self.fc1(x))  # Fully connected pertama + ReLU
        x = self.dropout(x)  # Dropout untuk mencegah overfitting
        x = F.relu(self.fc2(x))  # Fully connected kedua + ReLU
        x = self.fc3(x)  # Fully connected untuk output (tanpa aktivasi, langsung ke softmax nantinya)
        return x

## 6. Inisialisasi Model, Loss function, Optimizer, dan Learning Rate Scheduler

Kita bisa menggunakan **CrossEntropyLoss** untuk loss function dan menggunakan **Adam** untuk optimizer.

In [53]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Scheduler untuk learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

## 7. Fungsi Training

In [54]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Mengatur gradien menjadi nol
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Menghitung loss
        loss.backward()  # Backward pass (menghitung gradien)
        optimizer.step()  # Mengupdate parameter model
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    print(f'Train Loss: {running_loss / len(train_loader):.4f}, Accuracy: {100. * correct / total:.2f}%')

## 8. Model Testing

In [55]:
def test(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # Tidak perlu menghitung gradien selama testing
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    print(f'Test Loss: {test_loss / len(test_loader):.4f}, Accuracy: {100. * correct / total:.2f}%')

## 9. Validasi dan Evaluasi Model

In [56]:
for epoch in range(num_epochs):
    print(f'Epoch [{epoch+1}/{num_epochs}]')
    train(model, train_loader, criterion, optimizer, device)
    test(model, test_loader, criterion, device)
    
    # Update learning rate
    scheduler.step()

Epoch [1/100]
Train Loss: 1.6774, Accuracy: 36.97%
Test Loss: 1.3131, Accuracy: 52.45%
Epoch [2/100]
Train Loss: 1.2206, Accuracy: 55.89%
Test Loss: 1.0702, Accuracy: 61.26%
Epoch [3/100]
Train Loss: 1.0166, Accuracy: 63.98%
Test Loss: 0.9338, Accuracy: 66.87%
Epoch [4/100]
Train Loss: 0.8887, Accuracy: 69.06%
Test Loss: 0.7754, Accuracy: 72.92%
Epoch [5/100]
Train Loss: 0.8053, Accuracy: 72.09%
Test Loss: 0.7348, Accuracy: 74.66%
Epoch [6/100]
Train Loss: 0.7457, Accuracy: 74.14%
Test Loss: 0.7195, Accuracy: 74.81%
Epoch [7/100]
Train Loss: 0.6945, Accuracy: 75.98%
Test Loss: 0.6626, Accuracy: 77.19%
Epoch [8/100]
Train Loss: 0.6586, Accuracy: 77.17%
Test Loss: 0.6785, Accuracy: 76.43%
Epoch [9/100]
Train Loss: 0.6277, Accuracy: 78.50%
Test Loss: 0.6352, Accuracy: 77.66%
Epoch [10/100]
Train Loss: 0.6064, Accuracy: 79.05%
Test Loss: 0.6250, Accuracy: 77.63%
Epoch [11/100]
Train Loss: 0.5075, Accuracy: 82.41%
Test Loss: 0.5432, Accuracy: 81.16%
Epoch [12/100]
Train Loss: 0.4751, Accura